In [1]:
import warnings
warnings.filterwarnings("ignore")
from langchain.document_loaders import PyPDFDirectoryLoader

dir_cv = r'C:\Users\Gerardo Vilcamiza\Documents\Python Scripts\Master IA\LLMs\CVs'
file_loader = PyPDFDirectoryLoader(dir_cv)
doc_cv = file_loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
documents_cv = text_splitter.split_documents(doc_cv)

print(f"Total de fragmentos creados: {len(documents_cv)}")
print(f"Ejemplo de fragmento:\n{documents_cv[0].page_content[:500]}...")

Total de fragmentos creados: 10
Ejemplo de fragmento:
Gerardo Vilcamiza
Ingeniero Mecatrónico & Data Scientist
gerardo.vilcamiza@ieee.org
+51 951690621
Lima, Perú
linkedin.com/in/gerardo-vilcamiza/
github.com/gerarvilca02
0000-0001-6109-5515
PERFIL
Ingeniero Mecatrónico de la UPC, clasificado dentro del quinto superior de la carrera. Entusiasta y dedicado, a la
vanguardia de la tecnología y la investigación. Mi experticia abarca un amplio rango de softwares de diseño 2D y 3D,
lenguajes de programación, herramientas de inteligencia artificial y el m...


In [3]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

PINECONE_API_KEY = "pcsk_5o862d_Lphz9e8ANb6jeZTA2w7DHpjUHoGZKCLjCZWKEMKb1sVa7ofgvg9WFpjgWDpsp5Y"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'eguins'

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print("Índice {} borrado".format(index_name))

if index_name not in pc.list_indexes().names():
    print("Índice creado con el nombre: {}".format(index_name))
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric='cosine',
        spec=spec
    )
else:
    print("El índice con el nombre {} ya estaba creado".format(index_name))

Índice eguins borrado
Índice creado con el nombre: eguins


In [4]:
from langchain_pinecone import PineconeEmbeddings

embed_model = PineconeEmbeddings(model="multilingual-e5-large")

namespace = "espacio"

docsearch = PineconeVectorStore.from_documents(
    documents=documents_cv,
    index_name=index_name,
    embedding=embed_model,
    namespace=namespace
)

print("Valores insertados en el índice {}".format(index_name))
time.sleep(1)

Valores insertados en el índice eguins


In [5]:
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(model="llama3-70b-8192", temperature=0.2)

In [13]:
from langchain.chains import RetrievalQA

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

query = input("Introduce tu pregunta sobre los CVs cargados: ")

if query:
    response = qa_chain.run(query)
    print("\n**Respuesta:**\n")
    print(response)

Introduce tu pregunta sobre los CVs cargados:  Quién es Gerardo Vilcamiza?



**Respuesta:**

Gerardo Vilcamiza es un ingeniero mecatrónico y científico de datos (Data Scientist) con experiencia en inteligencia artificial, machine learning y desarrollo de proyectos de Big Data. También es un apasionado de la inteligencia artificial y la ciencia de datos, y ha trabajado en proyectos en Perú, Colombia y México.
